In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

unsorted_df = pd.read_csv('full.csv')

season_list = []
order_list = []
season = 15
order = 0
for line in unsorted_df.iterrows():
  if line[1][0] == 0:
    season += 1
    order = 0
  season_list.append(str(season) + '-' + str(season + 1))
  order_list.append(order)
  order += 1
unsorted_df['season'] = season_list
unsorted_df['order'] = order_list
df = unsorted_df.iloc[::-1].sort_values(by=['season', 'order'], ascending = [True, False])
df['order'] = np.arange(df.shape[0])
df.drop(df.columns[[0]], axis = 1, inplace = True)
df.drop(df.columns[[2,3, 10, 11]], axis = 1, inplace = True)
# достигнут хронологический порядок


labels_df = df[['thirdp_home', 'thirdp_away']]
labels_sum = df['thirdp_home'] + df['thirdp_away']
df.drop(df.columns[[6,7]], axis = 1, inplace = True)

'''
modified_trains = [df.iloc[0:1100]]
labels = [labels_df.iloc[0:1100]]
for i in range((df.shape[0] - 1100)//8):
  modified_trains.append(pd.concat([modified_trains[i], df.iloc[1100 +8*i: 1108 + 8*i]]))
  labels.append(pd.concat([labels[i],labels_df.iloc[1100 +8*i: 1108 + 8*i]]))
# спустя каждый тур (8 матчей) предсказанные результаты добавляются в train
'''
from catboost import CatBoostRegressor
from catboost import Pool

cat_features = ['season', 'team_home', 'team_away']
'''
#multiregression - catboost поддерживает только rmse, плохо
results = pd.DataFrame(['pred_home', 'pred_away', 'real_home', 'real_away'])
for i in range((df.shape[0] - 1100)//8 - 1):   #()
  model = CatBoostRegressor(iterations=1000, learning_rate=0.3, depth=3, loss_function='MultiRMSE')
  cat_train = Pool(modified_trains[i], labels[i], cat_features=cat_features)
  model.fit(cat_train, verbose=False)
  preds = model.predict(df.iloc[1100 +8*i: 1108 + 8*i])
'''
sum_modified_trains = [df.iloc[0:1100]]
sum_labels = [labels_sum.iloc[0:1100]]
loss_list = []
for i in range((df.shape[0] - 1100)//8):
  sum_modified_trains.append(pd.concat([sum_modified_trains[i], df.iloc[1100 +8*i: 1108 + 8*i]]))
  sum_labels.append(pd.concat([sum_labels[i],labels_sum.iloc[1100 +8*i: 1108 + 8*i]]))
# спустя каждый тур (~8 матчей) предсказанные результаты добавляются в train
for i in range((df.shape[0] - 1100)//8 - 1):   #()
  model = CatBoostRegressor(iterations=1000, learning_rate=0.3, depth=3, loss_function='RMSE')
  cat_train = Pool(sum_modified_trains[i], sum_labels[i], cat_features=cat_features)
  model.fit(cat_train, verbose=False)
  preds = model.predict(df.iloc[1100 +8*i: 1108 + 8*i])
  sums = (labels_df[1100 +8*i: 1108 + 8*i].sum(axis=1)).values
  print(preds)
  print(sums)
  loss_list.append(abs(preds - sums))

print(loss_list)
print(np.average(loss_list))

[36.70330755 48.54612008 41.29434601 38.20693099 35.62554324 40.86933437
 41.72969013 32.0103598 ]
[40 34 43 43 34 50 46 40]
[52.76268407 44.53539642 39.2536429  39.20212666 42.0626585  37.93772923
 42.08840075 43.60156706]
[32 34 34 57 36 41 32 36]
[41.04311506 36.72967977 42.56400706 46.18891481 40.42477501 37.15966782
 38.59350409 42.50596164]
[32 37 49 36 48 43 45 62]
[46.1256349  52.58805452 50.4356104  53.20180543 53.94801742 47.77569375
 48.73161817 53.38019565]
[34 51 47 49 29 35 39 47]
[37.5273176  47.35724548 49.37711825 41.75611784 45.77078165 47.58990489
 40.57507524 40.29870623]
[44 29 43 41 41 39 45 41]
[50.28816247 49.2449568  48.13846385 41.38932774 56.20109454 48.578088
 42.14334294 44.85852245]
[41 53 42 41 45 47 44 38]
[39.53293283 40.34970985 42.59121006 47.88654681 45.34588914 39.37504884
 47.00181121 47.45901337]
[40 32 50 34 56 46 35 40]
[41.11729759 39.16359159 32.40183179 39.48803609 44.72039256 38.93714297
 35.00277569 37.36336714]
[28 43 53 47 41 48 38 39]
[4